In [1]:
import pyspark.sql.functions as f

In [2]:
spark

In [3]:
df = spark.read.json("/home/ubuntu/temp-data")
df.show()
df.count()

+--------------+----+-------------------+----+------------------------------+---------------+
|extractionTime| geo|                 id|lang|                          text|           user|
+--------------+----+-------------------+----+------------------------------+---------------+
|    1639926584|null|1472584944698499073| und|          @Sen_JoeManchin ?...|     TopcatHuff|
|    1639926547|null|1472584789505028104| und|          @Sen_JoeManchin =...|    Retrievals1|
|    1639929022|null|1472595168285310981|  ja|@POTUS 私が大統領に就任して...|    Xmyqh597God|
|    1639928268|null|1472592005180301312|  ja|@POTUS どのクラスも卒業する...|    Xmyqh597God|
|    1639927408|null|1472588397696851969|  hi|          @JoeBiden #पढ़ी_ल...|  aao_deen_jane|
|    1639927413|null|1472588421310808068|  hi|          @POTUS @JoeBiden ...|  aao_deen_jane|
|    1639927382|null|1472588292486877184|  hi|          @HillaryClinton @...|  aao_deen_jane|
|    1639927395|null|1472588346874425345|  hi|          @MichelleObama @O...|  a

8219

In [4]:
df = df.withColumn("text", f.regexp_replace(f.col("text"), "[^A-Za-z0-9@ ]", ""))
df.show()

+--------------+----+-------------------+----+--------------------+---------------+
|extractionTime| geo|                 id|lang|                text|           user|
+--------------+----+-------------------+----+--------------------+---------------+
|    1639926584|null|1472584944698499073| und|@SenJoeManchin  h...|     TopcatHuff|
|    1639926547|null|1472584789505028104| und|    @SenJoeManchin  |    Retrievals1|
|    1639929022|null|1472595168285310981|  ja|    @POTUS 60021969 |    Xmyqh597God|
|    1639928268|null|1472592005180301312|  ja|@POTUS 2021 https...|    Xmyqh597God|
|    1639927408|null|1472588397696851969|  hi|@JoeBiden        ...|  aao_deen_jane|
|    1639927413|null|1472588421310808068|  hi|@POTUS @JoeBiden ...|  aao_deen_jane|
|    1639927382|null|1472588292486877184|  hi|@HillaryClinton @...|  aao_deen_jane|
|    1639927395|null|1472588346874425345|  hi|@MichelleObama @O...|  aao_deen_jane|
|    1639926916|null|1472586335487672330|  en|@SenJoeManchin On...|    Mikey

In [5]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [6]:
words = df.select(f.explode(f.split("text", " ")).alias("word"), "id")\
.select(f.lower("word").alias("word"), "id")\
.filter(~f.lower("word").isin(stopwords))

words.filter(~f.col("word").startswith("@")).groupby("word").count().orderBy(f.desc("count")).show()

+--------+-----+
|    word|count|
+--------+-----+
|        | 2656|
|      rt| 1563|
| manchin|  472|
|  people|  461|
|    must|  449|
|    back|  429|
|   party|  420|
|    vote|  388|
|     one|  386|
|     joe|  375|
|  better|  369|
|     get|  360|
|democrat|  350|
|   youre|  309|
|    dont|  302|
|   build|  301|
|    mask|  290|
|     bbb|  277|
|   would|  268|
| senator|  259|
+--------+-----+
only showing top 20 rows



In [7]:
df = words.groupby("id").agg(f.concat_ws(" ", f.collect_list("word")).alias("text"))
df.show()

+-------------------+--------------------+
|                 id|                text|
+-------------------+--------------------+
|1472582346029293578|rt @thedemocrats ...|
|1472582970590601217|@senblumenthal sa...|
|1472592728060055569|@lindseygrahamsc ...|
|1472589476530728960|@senjoemanchin @s...|
|1472583138752749573|@senjoemanchin ma...|
|1472594552892629002|rt @thedemocrats ...|
|1472596619975704576|@joebiden look re...|
|1472587458659463169|@senschumer manch...|
|1472587098456797188|@potus hypocritic...|
|1472593009199960069|@tedcruz wow sick...|
|1472583260945403911|@potus dont 300k ...|
|1472596424869220353|rt @tedcruz  offi...|
|1472593611543830530|@senjoemanchin @w...|
|1472583625405353988|@senjoemanchin gr...|
|1472597546874949633|@senjoemanchin of...|
|1472589004042350592|@senjoemanchin de...|
|1472592485708967941|rt @senwarren mak...|
|1472586136165961735|rt @senwarren mak...|
|1472585067058847755|@joemanchinwv tha...|
|1472583949562126346|rt @tedcruz  offi...|
+----------

In [8]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(df)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=200)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.limit(5).collect()

[Row(id=1472582346029293578, text='rt @thedemocrats battles must pass freedom vote act john lewis voting rights act must president', words=['rt', '@thedemocrats', 'battles', 'must', 'pass', 'freedom', 'vote', 'act', 'john', 'lewis', 'voting', 'rights', 'act', 'must', 'president'], rawFeatures=SparseVector(200, {14: 1.0, 23: 2.0, 46: 1.0, 57: 1.0, 69: 1.0, 101: 2.0, 106: 1.0, 128: 2.0, 132: 1.0, 183: 1.0, 190: 1.0, 192: 1.0}), features=SparseVector(200, {14: 2.2153, 23: 4.6519, 46: 3.3867, 57: 2.7471, 69: 3.5547, 101: 4.2823, 106: 2.8853, 128: 4.9236, 132: 2.8447, 183: 1.5188, 190: 3.0901, 192: 3.294})),
 Row(id=1472582970590601217, text='@senblumenthal say continue working referring communist party credentials p httpstcojl17bvmnka', words=['@senblumenthal', 'say', 'continue', 'working', 'referring', 'communist', 'party', 'credentials', 'p', 'httpstcojl17bvmnka'], rawFeatures=SparseVector(200, {62: 1.0, 90: 1.0, 91: 1.0, 143: 1.0, 161: 1.0, 168: 1.0, 169: 1.0, 172: 1.0, 179: 1.0, 184: 1

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, Normalizer, StopWordsRemover

In [11]:
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
stopwords_cleaner = StopWordsRemover(inputCol="tokens", outputCol="out")
nlp_pipeline = Pipeline(
    stages=[tokenizer,
            stopwords_cleaner])
nlp_model = nlp_pipeline.fit(df)
processed_df  = nlp_model.transform(df)
processed_df.show()

+-------------------+--------------------+--------------------+--------------------+
|                 id|                text|              tokens|                 out|
+-------------------+--------------------+--------------------+--------------------+
|1472582346029293578|rt @thedemocrats ...|[rt, @thedemocrat...|[rt, @thedemocrat...|
|1472582970590601217|@senblumenthal sa...|[@senblumenthal, ...|[@senblumenthal, ...|
|1472592728060055569|@lindseygrahamsc ...|[@lindseygrahamsc...|[@lindseygrahamsc...|
|1472589476530728960|@senjoemanchin @s...|[@senjoemanchin, ...|[@senjoemanchin, ...|
|1472583138752749573|@senjoemanchin ma...|[@senjoemanchin, ...|[@senjoemanchin, ...|
|1472594552892629002|rt @thedemocrats ...|[rt, @thedemocrat...|[rt, @thedemocrat...|
|1472596619975704576|@joebiden look re...|[@joebiden, look,...|[@joebiden, look,...|
|1472587458659463169|@senschumer manch...|[@senschumer, man...|[@senschumer, man...|
|1472587098456797188|@potus hypocritic...|[@potus, hypocrit...|[@

In [12]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="out", outputCol="features", vocabSize=500, minDF=3.0)
cv_model = cv.fit(processed_df)
vectorized_tokens = cv_model.transform(processed_df)
vectorized_tokens.show()

+-------------------+--------------------+--------------------+--------------------+--------------------+
|                 id|                text|              tokens|                 out|            features|
+-------------------+--------------------+--------------------+--------------------+--------------------+
|1472582346029293578|rt @thedemocrats ...|[rt, @thedemocrat...|[rt, @thedemocrat...|(500,[2,7,11,29,3...|
|1472582970590601217|@senblumenthal sa...|[@senblumenthal, ...|[@senblumenthal, ...|(500,[10,80,107,1...|
|1472592728060055569|@lindseygrahamsc ...|[@lindseygrahamsc...|[@lindseygrahamsc...|(500,[8,21,41,256...|
|1472589476530728960|@senjoemanchin @s...|[@senjoemanchin, ...|[@senjoemanchin, ...|(500,[0,1,5,111,1...|
|1472583138752749573|@senjoemanchin ma...|[@senjoemanchin, ...|[@senjoemanchin, ...|(500,[0,11,15,21,...|
|1472594552892629002|rt @thedemocrats ...|[rt, @thedemocrat...|[rt, @thedemocrat...|(500,[2,34,53,76,...|
|1472596619975704576|@joebiden look re...|[@jo

In [ ]:
from pyspark.ml.clustering import LDA
lda = LDA(k=5, maxIter=100)
model = lda.fit(vectorized_tokens)

In [ ]:
for indices in model.describeTopics(20).select("termIndices").rdd.flatMap(lambda x: x ).collect():
    print([cv_model.vocabulary[i] for i in indices])

In [ ]:
cv_model.vocabulary